# Задание #
Представьте: вы работаете в отделе продуктовой аналитики в ретейлере SkyLenta, задача от руководства — проанализировать результаты A/B-теста, чтобы принять обоснованное решение о смене маркетинговой механики.

Отдел маркетинга предложил новый способ воздействия на клиентские покупки — пуш-уведомления в приложении. До этого основным маркет. воздействием было уведомление с помощью баннера в приложении. Отдел маркетинга предполагает, что новый вид уведомлений эффективнее и с точки зрения конверсии в покупку, и с точки зрения среднего чека.

Чтобы найти лучший способ, было проведено A/B-тестирование в городах присутствия ритейлера в России. Контрольной группе по-прежнему присылали уведомления в форме баннера, а тестовой — в форме пуш-уведомления.

Эксперимент продолжался 3 месяца.

Сплит-система: клиенты разбиты на две группы одинакового размера случайным образом.
Были исследованы две таргет-метрики: конверсия из рекламы в покупку и средний чек покупки.

## Задача ##

Вам необходимо поработать с результатами A/B-теста:
- Очистить данные и подготовить их к дальнейшему анализу.
- Проанализировать результаты A/B-тестов и подготовить визуализации.
- Провести сегментацию по городам и торговым точкам и рассчитать результаты A/B-теста на каждом сегменте.
- Сделать выводы и оформить результаты исследования в форме удобного Excel-калькулятора.

### Подготовка ###

In [ ]:
### Импорт библиотек и данных

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu, ttest_ind

In [ ]:

new_dipl_ds = pd.read_excel('Новый диплом датасет.xlsx', sheet_name = None)
data = new_dipl_ds['Данные']
clients = new_dipl_ds['Clients']
regions = new_dipl_ds['Region_dict']

In [ ]:
### Проверка, очистка и подготовка данных

regions.info()

In [ ]:

regions.duplicated().sum()

In [ ]:

clients.info()

In [ ]:

clients.duplicated().sum()

In [ ]:

clients.head()

In [ ]:
### отсортируем по возрастанию id_client, сбросим старые индексы и внесем новые в отсортированном по возрастанию порядке

clients.sort_values('id_client', inplace = True)
clients.reset_index(inplace = True)
clients.drop('index', axis = 1, inplace = True)
clients.head(94)

In [ ]:
### Отсортировав обнаружили, что рекламные объявляния показывались последовательно по нарастанию порядкового номера id_client
### чтобы сохранить данные для тестирования, заполнили "нулы" в колонке dtime_ad предыдущими по порядку значениями.

clients['dtime_ad'] = clients['dtime_ad'].ffill()
clients.info()

In [ ]:

data.info()

In [ ]:

data.duplicated().sum()

In [ ]:

data.isnull().sum()

In [ ]:
### удалим "нулы" во всем датафрейме

data = data[~data['dtime_pay'].isnull()]
data.info()

In [ ]:

print()
print('Датафрейм regions не содержит "нулов" и дубликатов, форматы данных соответствуют необходимым')
print()
regions.head()

- id_trading_point — уникальный идентификатор торговой точки, к которой прикреплен данный пользователь.
- city — название города.

In [ ]:

print()
print(f'Датафрейм clients не содержит дубликатов, форматы данных соответствуют необходимым. Отсортировав датарейм по возрастанию id_client обнаружили, что рекламные объявляния показывались последовательно по нарастанию порядкового номера id_client, соответственно, чтобы не удалять, а сохранить данные для тестирования, заполнили "нулы" в колонке dtime_ad предыдущими по порядку значениями.')
print()
clients.head()

- id_client — уникальный идентификатор клиента.
- dtime_ad — дата и время показа рекламного объявления (или баннером в приложении, или пуш-уведомлением, в зависимости от группы).
- nflag_test — группа эксперимента (0 — контроль, 1 — тест).
- id_trading_point — идентификатор торговой точки, к которой прикреплен данный пользователь.

In [ ]:

print()
print('Датафрейм regions не содержит "нулов" и дубликатов, форматы данных соответствуют необходимым. Сортировка по dtime_pay, id_client и id_order не показала возможности заполнить пропущенные значения в данных.')
print()
data.head()

- id_order — уникальный идентификатор покупки.
- id_client — уникальный идентификатор клиента.
- amt_payment — размер платежа.
- dtime_pay — дата и время оплаты.

### Решение ###

In [ ]:
### Выведем кол-во торговых точек по городам, визуализируем

tr_p_city = regions.groupby('city').agg(cnt_tr_p = ('id_trading_point', 'count')).sort_values('cnt_tr_p', ascending = False).reset_index()
print('Кол-во городов присутствия ритейлера SkyLenta', regions['city'].nunique())
print()
tr_p_city.head(25)

In [ ]:

fig, ax = plt.subplots(figsize=(10, 5))

plt.bar(tr_p_city['city'], tr_p_city['cnt_tr_p'], alpha = 0.3, color = 'green')
plt.title('Распределение торговых точек по городам присутствия')
plt.xlabel('Город присутствия')
plt.ylabel('Кол-во торговых точек')
plt.xticks(rotation = 45, ha = 'right')
plt.grid(axis = 'y', linestyle = '--', alpha = 0.5)
plt.show()

In [ ]:
### Объединим таблицы

data_agg = data.groupby('id_client').agg(sum_paym = ('amt_payment', 'sum')).reset_index()
print(f'Кол-во уникальных id_client в исходном датафрейме data - {data['id_client'].nunique()}')
print()
print(f'Кол-во уникальных id_client в агрегированном датафрейме data_agg - {data_agg['id_client'].nunique()}')
print()
data_agg.head()

In [ ]:

df = clients.merge(data_agg, on = 'id_client', how = 'left')
df.info()

In [ ]:
### Заполнили на "0" пустые ячейки в столбце sum_paym, чтобы обеспечить возможность работы с ними

df['sum_paym'] = df['sum_paym'].fillna(0)
df.isnull().sum()

In [ ]:

df = df.merge(regions, on = 'id_trading_point', how = 'left')
df.info()

In [ ]:

df.head()

In [ ]:
### Добавляем к таблице столбец flag_paym, отображающий наличие/отсутствие платежей по клиенту

df['flag_paym'] = np.where(df['sum_paym'] > 0, 1, 0)
df.head()

In [ ]:
### Создаем функции для автоматизации вычислений

def ttest_calc(r1, r2, alpha = 0.05):
    c_tt, p_value_tt = ttest_ind(r1, r2)
    print(f'Значение критерия Стьюдента = {c_tt}.')
    print()
    if p_value_tt >= alpha:
        print(f'p value = {p_value_tt}. Принимаем H0, т.е. не обнаружено стат. значимой разницы между выборками.')
    else:
        print(f'p value = {p_value_tt}. Принимаем H1, т.е. обнаружена стат. значимая разница между выборками.')
    print()

In [ ]:

def mann_whitney_func(r1, r2, alpha = 0.05):
    c_mw, p_value_mw = mannwhitneyu(r1, r2)
    print(f'Значение критерия Манна-Уитни = {c_mw}, отражает разницу в распределениях двух выборок.')
    print()
    if p_value_mw >= alpha:
        print(f'p value = {p_value_mw}. Н0 отклоняется, указывая на статистически значимые различия между двумя выборками.')
    else:
        print(f'p value = {p_value_mw}. Нет достаточных оснований отклонять Н0 и нельзя утверждать, что распределения различаются.')
    print()

In [ ]:
### Создаем списки "пустых" торговых точек

df_tr_point_pays = df.groupby(['id_trading_point', 'nflag_test']).agg(sum_paym = ('sum_paym', 'sum'), cnt_clt = ('id_client', 'count')).reset_index()

tr_point_without_pay = df_tr_point_pays[df_tr_point_pays['sum_paym'] == 0]['id_trading_point'].unique().tolist()
print()
print(f'{tr_point_without_pay} - cписок торговых точек, в которых в тестовой или в контрольной группе не было заплачено ни одного рубля ни одним клиентом.')
print()

groups_per_point = df_tr_point_pays.groupby('id_trading_point')['nflag_test'].nunique()
one_group_tr_points = groups_per_point[groups_per_point == 1].index.tolist()
print(f'{one_group_tr_points} - cписок торговых точек, в которых клиенты только из одной группы, т.е. клиентов второй группы нет.')
print()

tr_point_without_group = df_tr_point_pays[((df_tr_point_pays['nflag_test'] == 0) & (df_tr_point_pays['cnt_clt'] == 0)) | ((df_tr_point_pays['nflag_test'] == 1) & (df_tr_point_pays['cnt_clt'] == 0))]['id_trading_point'].unique().tolist()
print(f'{tr_point_without_group} - проверка на торговые точки, в которых обозначились 2 группы, но одна из них пустая.')
print()

In [ ]:
### Удаляем из датафрейма данные по торговым точкам с нулевыми платежами и нулевыми группами.

print(f'Кол-во торговых точек до удаления - {df['id_trading_point'].nunique()}')
print()
all_0_tr_points = tr_point_without_pay + one_group_tr_points + tr_point_without_group
df = df[~df['id_trading_point'].isin(all_0_tr_points)].dropna()
print(f'Кол-во торговых точек после удаления - {df['id_trading_point'].nunique()}')
print()

### Результаты А/В-тестов по общим выборкам (0 — контроль, 1 — тест). ###

In [ ]:

fig = plt.figure(figsize=(10, 5))

plt.hist(df[df['nflag_test'] == 0]['sum_paym'], color = 'g', label = 'Контрольная группа', alpha = 0.4)
plt.hist(df[df['nflag_test'] == 1]['sum_paym'], color = 'gold', label = 'Тестовая группа', alpha = 0.4)
plt.title('Распределение платежей по группам клиентов')
plt.xlabel('Размер платежа')
plt.ylabel('Кол-во платежей')
#plt.xlim(0, 20000)
plt.legend(loc = 'upper right')
plt.show()

In [ ]:
### Посмотрим, каковы значения в каждой выборке

print()
print(f'Контрольная выборка - {df[df['nflag_test'] == 0]['sum_paym'].describe()}')
print()
print(f'Тестовая выборка - {df[df['nflag_test'] == 1]['sum_paym'].describe()}')
print()

In [ ]:
### сравниваем средние платежи

r1 = df[df['nflag_test'] == 0]['sum_paym']
r2 = df[df['nflag_test'] == 1]['sum_paym']

print()
print('Сравнение по средним платежам в разбивке по группам')
print()
print()
ttest_calc(r1, r2, alpha = 0.05)
print()
mann_whitney_func(r1, r2, alpha = 0.05)

In [ ]:
### сравниваем конверсию в платеж

r1 = df[df['nflag_test'] == 0]['flag_paym']
r2 = df[df['nflag_test'] == 1]['flag_paym']

print()
print('Сравнение средних конверсий в оплату в разбивке по группам')
print()
print()
ttest_calc(r1, r2, alpha = 0.05)
print()
mann_whitney_func(r1, r2, alpha = 0.05)

### Результаты А/В-тестов по выборкам (0 — контроль, 1 — тест) для Москвы. ###

In [ ]:

fig = plt.figure(figsize=(10, 5))

plt.hist(df[(df['nflag_test'] == 0) & (df['city'] == 'Москва')]['sum_paym'], color = 'blue', label = 'Контрольная группа', alpha = 0.4)
plt.hist(df[(df['nflag_test'] == 1) & (df['city'] == 'Москва')]['sum_paym'], color = 'red', label = 'Тестовая группа', alpha = 0.4)
plt.title('Распределение платежей по группам клиентов')
plt.xlabel('Размер платежа')
plt.ylabel('Кол-во платежей')
#plt.xlim(0, 20000)
plt.legend(loc = 'upper right')
plt.show()

In [ ]:
### Посмотрим, каковы значения в каждой выборке

print()
print(f'Контрольная выборка - {df[(df['nflag_test'] == 0) & (df['city'] == 'Москва')]['sum_paym'].describe()}')
print()
print(f'Тестовая выборка - {df[(df['nflag_test'] == 1) & (df['city'] == 'Москва')]['sum_paym'].describe()}')
print()

In [ ]:
### сравниваем средние платежи

r1 = df[(df['nflag_test'] == 0) & (df['city'] == 'Москва')]['sum_paym']
r2 = df[(df['nflag_test'] == 1) & (df['city'] == 'Москва')]['sum_paym']

print()
print('Сравнение по средним платежам в разбивке по группам')
print()
print()
ttest_calc(r1, r2, alpha = 0.05)
print()
mann_whitney_func(r1, r2, alpha = 0.05)

In [ ]:
### сравниваем конверсию в платеж

r1 = df[(df['nflag_test'] == 0) & (df['city'] == 'Москва')]['flag_paym']
r2 = df[(df['nflag_test'] == 1) & (df['city'] == 'Москва')]['flag_paym']

print()
print('Сравнение средних конверсий в оплату в разбивке по группам')
print()
print()
ttest_calc(r1, r2, alpha = 0.05)
print()
mann_whitney_func(r1, r2, alpha = 0.05)

### Результаты А/В-тестов по выборкам (0 — контроль, 1 — тест) для Санкт-Петербурга. ###

In [ ]:

fig = plt.figure(figsize=(10, 5))

plt.hist(df[(df['nflag_test'] == 0) & (df['city'] == 'Санкт-Петербург')]['sum_paym'], color = 'y', label = 'Контрольная группа', alpha = 0.5)
plt.hist(df[(df['nflag_test'] == 1) & (df['city'] == 'Санкт-Петербург')]['sum_paym'], color = 'm', label = 'Тестовая группа', alpha = 0.3)
plt.title('Распределение платежей по группам клиентов')
plt.xlabel('Размер платежа')
plt.ylabel('Кол-во платежей')
#plt.xlim(0, 20000)
plt.legend(loc = 'upper right')
plt.show()

In [ ]:
### Посмотрим, каковы значения в каждой выборке

print()
print(f'Контрольная выборка - {df[(df['nflag_test'] == 0) & (df['city'] == 'Санкт-Петербург')]['sum_paym'].describe()}')
print()
print(f'Тестовая выборка - {df[(df['nflag_test'] == 1) & (df['city'] == 'Санкт-Петербург')]['sum_paym'].describe()}')
print()

In [ ]:
### сравниваем средние платежи

r1 = df[(df['nflag_test'] == 0) & (df['city'] == 'Санкт-Петербург')]['sum_paym']
r2 = df[(df['nflag_test'] == 1) & (df['city'] == 'Санкт-Петербург')]['sum_paym']

print()
print('Сравнение по средним платежам в разбивке по группам')
print()
print()
ttest_calc(r1, r2, alpha = 0.05)
print()
mann_whitney_func(r1, r2, alpha = 0.05)

In [ ]:
### сравниваем конверсию в платеж

r1 = df[(df['nflag_test'] == 0) & (df['city'] == 'Санкт-Петербург')]['flag_paym']
r2 = df[(df['nflag_test'] == 1) & (df['city'] == 'Санкт-Петербург')]['flag_paym']

print()
print('Сравнение средних конверсий в оплату в разбивке по группам')
print()
print()
ttest_calc(r1, r2, alpha = 0.05)
print()
mann_whitney_func(r1, r2, alpha = 0.05)

### Результаты А/В-тестов по выборкам (0 — контроль, 1 — тест) для других городов. ###

In [ ]:

fig = plt.figure(figsize=(10, 5))

plt.hist(df[(df['nflag_test'] == 0) & (df['city'] != 'Санкт-Петербург') & (df['city'] != 'Москва')]['sum_paym'], color = 'y', label = 'Контрольная группа', alpha = 0.5)
plt.hist(df[(df['nflag_test'] == 1) & (df['city'] != 'Санкт-Петербург') & (df['city'] != 'Москва')]['sum_paym'], color = 'm', label = 'Тестовая группа', alpha = 0.3)
plt.title('Распределение платежей по группам клиентов')
plt.xlabel('Размер платежа')
plt.ylabel('Кол-во платежей')
#plt.xlim(0, 20000)
plt.legend(loc = 'upper right')
plt.show()

In [ ]:
### Посмотрим, каковы значения в каждой выборке

print()
print(f'Контрольная выборка - {df[(df['nflag_test'] == 0) & (df['city'] != 'Санкт-Петербург') & (df['city'] != 'Москва')]['sum_paym'].describe()}')
print()
print(f'Тестовая выборка - {df[(df['nflag_test'] == 1) & (df['city'] != 'Санкт-Петербург') & (df['city'] != 'Москва')]['sum_paym'].describe()}')
print()

In [ ]:
### сравниваем средние платежи

r1 = df[(df['nflag_test'] == 0) & (df['city'] != 'Санкт-Петербург') & (df['city'] != 'Москва')]['sum_paym']
r2 = df[(df['nflag_test'] == 1) & (df['city'] != 'Санкт-Петербург') & (df['city'] != 'Москва')]['sum_paym']

print()
print('Сравнение по средним платежам в разбивке по группам')
print()
print()
ttest_calc(r1, r2, alpha = 0.05)
print()
mann_whitney_func(r1, r2, alpha = 0.05)

In [ ]:
### сравниваем конверсию в платеж

r1 = df[(df['nflag_test'] == 0) & (df['city'] != 'Санкт-Петербург') & (df['city'] != 'Москва')]['flag_paym']
r2 = df[(df['nflag_test'] == 1) & (df['city'] != 'Санкт-Петербург') & (df['city'] != 'Москва')]['flag_paym']

print()
print('Сравнение средних конверсий в оплату в разбивке по группам')
print()
print()
ttest_calc(r1, r2, alpha = 0.05)
print()
mann_whitney_func(r1, r2, alpha = 0.05)

### Создадим датафрейм для дашборда ###

In [ ]:

listt = []

alpha = 0.05
cnt_df_clients = df['id_client'].count()

for trad_p in df['id_trading_point'].unique():
    df_city_tradp = df[df['id_trading_point'] == trad_p]

    city = df_city_tradp['city'].iloc[0]
    
    count_test = len(df_city_tradp[df_city_tradp['nflag_test']==1])
    count_control = len(df_city_tradp[df_city_tradp['nflag_test']==0])
        
    count_all = count_test + count_control
        
    percent_count = count_all / cnt_df_clients * 100

    sample1 = df_city_tradp[df_city_tradp['nflag_test'] == 1]['sum_paym']
    sample0 = df_city_tradp[df_city_tradp['nflag_test'] == 0]['sum_paym']
        
    avg_payment_test = sample1.mean()
    avg_payment_control = sample0.mean()

    diff = avg_payment_test - avg_payment_control 

    sigma_test = np.std(sample1, ddof = 1)
    sigma_control = np.std(sample0, ddof = 1)

    c_tt, p_value_tt = ttest_ind(sample1, sample0)

    result = np.where((p_value_tt < alpha) & (diff > 0),'positive', np.where((p_value_tt < alpha) & (diff < 0), 'negative', 'neutral'))

    listt.append({'city':city, 'id_trading_point':trad_p, 'count_test':count_test, 'count_control':count_control, 'count_all':count_all, 
                  'percent_count':percent_count, 'avg_payment_test':avg_payment_test, 'avg_payment_control':avg_payment_control, 'diff':diff, 
                  'sigma_test':sigma_test, 'sigma_control':sigma_control, 'ttest':c_tt, 'pvalue_ttest':p_value_tt, 'exp_flag':result})

to_excel = pd.DataFrame(listt)
to_excel.head()

In [ ]:

to_excel.info()

In [ ]:

#to_excel.to_excel('Для дашборда.xlsx', index = True)

In [ ]:

categories = ['positive', 'negative', 'neutral']

with pd.ExcelWriter('Для дашборда.xlsx') as writer:
    for category in categories:
        df_filtered = to_excel[to_excel['exp_flag'] == category]
        
        # Используем значение категории как название листа
        # Ограничиваем длину названия листа до 31 символа (ограничение Excel)
        sheet_name = str(category)[:31]
        
        df_filtered.to_excel(writer, sheet_name = sheet_name, index=False)